**This tutorial is based on/copied from http://peterbloem.nl/blog/transformers.**

# Imports

In [ ]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
from keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
from torch.utils.data import TensorDataset, DataLoader
from torch.optim import Adam

!pip install pytorch_lightning
import pytorch_lightning as pl

# Load the TensorBoard notebook extension.
%load_ext tensorboard

# Transformer Implementation - Copied from *Transformer* Notebook
The transformer architecture consists of multiple transformer blocks that typically look like this: 

<img src="https://raw.githubusercontent.com/leox1v/dl20/b3d5b5556d1b2bd360a4abeef4fd82f056ab0301/imgs/transformer-block.svg?token=AD5WN2SZYWM6XGH5SXMZM7S7VU3H4" alt="drawing" width="500"/>


It combines a self attention layer, [layer normalization](https://pytorch.org/docs/stable/generated/torch.nn.LayerNorm.html), a feed forward layer and another layer normalization. Additionally, it uses residual connections around the self attention and feed forward layer.

In [ ]:
# Let's implement a SelfAttention torch module.

class SelfAttention(nn.Module):
    """
    A SelfAttention model.
    Args:
        d: The embedding dimension.
        heads: The number of attention heads.
    """
    def __init__(self, d: int, heads: int=8):
        super().__init__()
        self.k, self.h = d, heads
        
        self.Wq = nn.Linear(d, d * heads, bias=False)
        self.Wk = nn.Linear(d, d * heads, bias=False)
        self.Wv = nn.Linear(d, d * heads, bias=False)
        
        # This unifies the outputs of the different heads into 
        # a single k-dimensional vector
        self.unifyheads = nn.Linear(heads * d, d)
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Args:
            x: The input embedding of shape [b, l, d].
            
        Returns:
            Self attention tensor of shape [b, l, d].
        """
        b, l, d = x.size()
        h = self.h
        
        # Transform the input embeddings x of shape [b, l, d] to queries, keys, values.
        # The output shape is [b, l, d, d*h] which we transform into [b, l, h, d]. Then,
        # we fold the heads into the batch dimenstion to arrive at [b*h, l, d]
        queries = self.Wq(x).view(b, l, h, d).transpose(1, 2).contiguous().view(b * h, l, d)
        keys = self.Wk(x).view(b, l, h, d).transpose(1, 2).contiguous().view(b * h, l, d)
        values = self.Wv(x).view(b, l, h, d).transpose(1, 2).contiguous().view(b * h, l, d)
        
        # Compute the product of queries and keys and scale with sqrt(d).
        # The tensor w' has shape (b*h, l, l) containing raw weights.
        w_prime = torch.bmm(queries, keys.transpose(1, 2)) / np.sqrt(d)

        # Compute w by normalizing w' over the last dimension.
        w = F.softmax(w_prime, dim=-1) 
        
        # Apply the self attention to the values.
        out = torch.bmm(w, values).view(b, h, l, d)
        
        # Swap h, l back.
        out = out.transpose(1, 2).contiguous().view(b, l, h * d)
        
        # Unify heads to arrive at shape [b, l, d].
        return self.unifyheads(out)
  
  
class TransformerBlock(nn.Module):
    """
    A Transformer block consisting of self attention and ff-layer.
    
    Args:
        d (int): The embedding dimension.
        heads (int): The number of attention heads.
        n_mlp (int): The number of mlp 'blocks'.
    """
    def __init__(self, d: int, heads: int=8, n_mlp: int=4):
        super().__init__()
        
        # The self attention layer.
        self.attention = SelfAttention(d, heads=heads)
        
        # The two layer norms.
        self.norm1 = nn.LayerNorm(d)
        self.norm2 = nn.LayerNorm(d)
        
        # The feed-forward layer.
        self.ff = nn.Sequential(
            nn.Linear(d, n_mlp*d),
            nn.ReLU(),
            nn.Linear(n_mlp*d, d)
        )
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Args:
            x: The input embedding of shape [b, l, d].
            
        Returns:
            Transformer output tensor of shape [b, l, d].
        """
        x_prime = self.attention(x)
        x = self.norm1(x_prime + x)
        
        x_prime = self.ff(x)
        return self.norm2(x_prime + x)


# IMDB Movie Reviews Sentiment Classification
We train a Transformer classification model to predict the sentiment (positive, negative) of movie reviews in the [IMDB dataset](https://keras.io/api/datasets/imdb/).

We use the [pytorch lightning framework](https://pytorch-lightning.readthedocs.io/en/latest/) to massively reduce the amount of code we need to write.

<img src="https://raw.githubusercontent.com/leox1v/dl20/b3d5b5556d1b2bd360a4abeef4fd82f056ab0301/imgs/classifier.svg?token=AD5WN2R4NHQQTQFOSQHAVWK7VU3JU" alt="drawing" width="800"/>

## Data Loader

In [ ]:
class IMDBDataModule(pl.LightningDataModule):
    """
    LightningDataModule to load the IMDB movie review sentiment data.

    Args:
        batch_size (int): The batch size for the train, test, and val 
                            dataloader.
    """ 
    
    def __init__(self, batch_size: int):
        super().__init__()
        self.batch_size = batch_size
        
    def setup(self, num_words: int, max_seq_len: int):
        """
        Initial loading of the dataset and transformation.
        
        Args:
            num_words (int): The vocabulary size. The vocabulary is 
                sorted by frequency of appearance in the dataset.
            max_seq_len (int): The maximum number of tokens per
                review.
        """
        (self.x_train, self.y_train), (self.x_test, self.y_test) = imdb.load_data(
            num_words=num_words, 
            maxlen=max_seq_len
        )
        print(f'# Training Examples: {len(self.y_train)}')
        print(f'# Test Examples: {len(self.y_test)}')
        
        self.word2idx = dict(
            **{k: v+3 for k, v in imdb.get_word_index().items()},
            **{'<PAD>': 0,
               '<START>': 1,
               '<UNK>': 2,
               '<UNUSED>': 3,
              },
        )
        self.idx2word = {v: k for k, v in self.word2idx.items()}
        
        # Pad the inputs and convert to torch Tensors.
        self.x_train = pad_sequences(self.x_train, maxlen=max_seq_len, value = 0.0)
        self.x_test = pad_sequences(self.x_test, maxlen=max_seq_len, value = 0.0)
        
    
    def example(self):
        """Returns a random training example."""        
        idx = np.random.randint(0, len(self.x_train))
        x, y = self.x_train[idx], self.y_train[idx]
        review = ' '.join(self.idx2word[token_id] for token_id in x if token_id > 1)
        sentiment = 'POSITIVE' if y else 'NEGATIVE'
        return f'{review}\nSentiment: {sentiment}'
    
    def train_dataloader(self):
        dataset = TensorDataset(torch.LongTensor(self.x_train), 
                                torch.LongTensor(self.y_train))
        return DataLoader(dataset, self.batch_size)
                                
    def test_dataloader(self):
        dataset = TensorDataset(torch.LongTensor(self.x_test), 
                                torch.LongTensor(self.y_test))
        return DataLoader(dataset, self.batch_size)
    
    def val_dataloader(self):
        return self.test_dataloader()
    
imdb_data = IMDBDataModule(128)
imdb_data.setup(num_words=30000,
                max_seq_len=100)
print('\nExamples:')
print('\n\n'.join(imdb_data.example() for _ in range(3)))

## Transformer Classification Model

In [ ]:
class TextClassificationTransformer(pl.LightningModule):
    """
    Stacked Transformer blocks for sequence classification.
    
    Args:
        d (int): The embedding dimension.
        heads (int): The number of attention heads for each transformer block.
        depth (int): The number of transformer blocks.
        max_seq_len (int): The maximum number of tokens of each sequence.
        num_tokens (int): The vocabulary size.
        num_classes (int): The number of classification classes.
        learning_rate (float): The learning rate for the optimizer.
    """
    def __init__(self, d: int=128, heads: int=8, depth: int=6,
                max_seq_len: int=512, num_tokens: int=30000, 
                num_classes: int=2, learning_rate: float=1e-4):
        super().__init__()
        # Save arguments in self.hparams.
        self.save_hyperparameters()
        
        self.num_tokens = num_tokens
        
        # Embeddings for tokens and position.
        self.token_emb = nn.Embedding(num_tokens, d)
        self.pos_emb = nn.Embedding(max_seq_len, d)
        
        # The stacked transformer blocks.
        self.transformer_blocks = nn.Sequential(
            *[TransformerBlock(d=d, heads=heads) for _ in range(depth)]
        )
        
        # Mapping of final output sequence to class logits.
        self.classification = nn.Linear(d, num_classes)
        
        self.criterion = nn.CrossEntropyLoss()
        self.accuracy = pl.metrics.Accuracy()
        
    def forward(self, x: torch.LongTensor) -> torch.FloatTensor:
        """
        Args:
            x (torch.LongTensor): A tensor of shape (b, l) of long values
                representing words in some predetermined vocabulary.
        
        Returns:
            A tensor of shape (b, c) of logits over the classes
                (c is the number of classes).
        """
        # 1. Generate token embeddings. Shape: [b, l, d].
        # 2. Generate position embeddings. Shape: [b, l, d].
        # 3. Generate final embedding by taking the sum of the two embeddings.
        #----------------
        # TODO
        #----------------
        
        # 4. Feed the embedding into the transformer blocks. Shape: [b, l, d].
        # 5. Compute the mean latent vector for each sequence.
        #    The mean is applied over dim=1 (time). Shape: [b, d].
        # 6. Classify. Shape: [b, num_classes].
        #----------------
        # TODO
        #----------------
        return out

    
    def configure_optimizers(self):
        """Specify the optimizer used for training."""
        return Adam(self.parameters(), lr=self.hparams.learning_rate)
    
    def training_step(self, batch, batch_idx):
        """A single training step.
        
        Args:
            batch (List(torch.LongTensor, torch.LongTensor)): The current batch
                inputs x and sentiment predictions y from the train dataloader.
            batch_idx (int): The index of the current batch.
        
        Returns:
            The loss for the current batch.
        """
        x, y = batch
        
        # Forward pass.
        logits = self(x)
        
        # Compute the loss with CrossEntropy.
        loss = self.criterion(logits, y)
        
        # Log the metrics.
        self.log('loss', loss, on_epoch=True, prog_bar=True)
        self.log('acc', self.accuracy(logits, y), on_epoch=True, prog_bar=True)
        return loss
    
    def test_step(self, batch, batch_idx):
        """A single test step.

        Pytorch Lightning automatically disable gradients and puts model in eval
        mode.
        
        Args:
            batch (List(torch.LongTensor, torch.LongTensor)): The current batch
                inputs x and sentiment predictions y from the *test* dataloader.
            batch_idx (int): The index of the current batch.
        
        Returns:
            The loss for the current batch.
        """
        x, y = batch
        logits = self(x)
        loss = self.criterion(logits, y)
        
        # Log the metrics.
        self.log('test_loss', loss, on_epoch=True)
        self.log('test_acc', self.accuracy(logits, y), on_epoch=True,
                 prog_bar=True)
        
    def validation_step(self, batch, batch_idx):
        """A single validation step.

        Since we don't have a seperate validation set we just use the test
        logic.
        
        Args:
            batch (List(torch.LongTensor, torch.LongTensor)): The current batch
                inputs x and sentiment predictions y from the *val* dataloader.
            batch_idx (int): The index of the current batch.
        
        Returns:
            The loss for the current batch.
        """
        return self.test_step(batch, batch_idx)

### Train Transformer Classification Model

In [ ]:
NUM_WORDS = 10000
MAX_SEQ_LEN = 128
EMBEDDING_DIM = 128
BATCH_SIZE = 32

imdb_data = IMDBDataModule(batch_size=BATCH_SIZE)
imdb_data.setup(num_words=NUM_WORDS,
                max_seq_len=MAX_SEQ_LEN)

model = TextClassificationTransformer(d=EMBEDDING_DIM,
                                      max_seq_len=MAX_SEQ_LEN,
                                      num_tokens=NUM_WORDS)
logger = pl.loggers.TensorBoardLogger('tb_logs', name='transformer')
trainer = pl.Trainer(max_epochs=5,
                     default_root_dir='ckpts',
                     gpus=1,
                     logger=logger)
trainer.fit(model, imdb_data)
_ = trainer.test()

## RNN Classification Model
Let's compare our Transformer model against an RNN. We want to build a basic stacked RNN architecture as shown in the image below, where we use the output of the final cell to predict the sentiment label. 

<img src="https://github.com/bentrevett/pytorch-sentiment-analysis/blob/master/assets/sentiment4.png?raw=true" alt="drawing" width="600"/>

In [ ]:
class TextClassificationRNN(pl.LightningModule):
    """
    RNN for sequence classification.
    
    Args:
        d (int): The embedding dimension.
        depth (int): The number of RNN blocks.
        max_seq_len (int): The maximum number of tokens of each sequence.
        num_tokens (int): The vocabulary size.
        num_classes (int): The number of classification classes.
        learning_rate (float): The learning rate for the optimizer.
    """
    def __init__(self, d: int=128, depth: int=6,
                max_seq_len: int=512, num_tokens: int=30000, 
                num_classes: int=2, learning_rate: float=1e-4):
        super().__init__()
        # Save arguments in self.hparams.
        self.save_hyperparameters()
        
        self.num_tokens = num_tokens
        
        # Embeddings for tokens.
        self.token_emb = nn.Embedding(num_tokens, d)
        
        # The stacked GRU layers.
        self.rnn = nn.GRU(input_size=d,
                          hidden_size=d,
                          num_layers=depth,
                          batch_first=True)
        
        # Mapping of final output sequence to class logits.
        self.classification = nn.Linear(d, num_classes)
        
        self.criterion = nn.CrossEntropyLoss()
        self.accuracy = pl.metrics.Accuracy()
        
    def forward(self, x: torch.LongTensor) -> torch.FloatTensor:
        """
        Args:
            x (torch.LongTensor): A tensor of shape (b, l) of long values
                representing words in some predetermined vocabulary.
        
        Returns:
            A tensor of shape (b, c) of logits over the classes
                (c is the number of classes).
        """
        # 1. Generate token embeddings (No position embedding required for RNNs). 
        #     Shape: [b, l, d].
        # 2. Feed the embedding into the GRU. Shape: [b, l, d]. Use the output
        #     of the last token as the encoding.
        # 3. Classify. Shape: [b, num_classes].
        #----------------
        # TODO
        #----------------
        return out

    def configure_optimizers(self):
        """Specify the optimizer used for training."""
        return Adam(self.parameters(), lr=self.hparams.learning_rate)
    
    def training_step(self, batch, batch_idx):
        """A single training step.
        
        Args:
            batch (List(torch.LongTensor, torch.LongTensor)): The current batch
                inputs x and sentiment predictions y from the train dataloader.
            batch_idx (int): The index of the current batch.
        
        Returns:
            The loss for the current batch.
        """
        x, y = batch
        
        # Forward pass.
        logits = self(x)
        
        # Compute the loss with CrossEntropy.
        loss = self.criterion(logits, y)
        
        # Log the metrics.
        self.log('loss', loss, on_epoch=True, prog_bar=True)
        self.log('acc', self.accuracy(logits, y), on_epoch=True, prog_bar=True)
        return loss
    
    def test_step(self, batch, batch_idx):
        """A single test step.

        Pytorch Lightning automatically disable gradients and puts model in eval
        mode.
        
        Args:
            batch (List(torch.LongTensor, torch.LongTensor)): The current batch
                inputs x and sentiment predictions y from the *test* dataloader.
            batch_idx (int): The index of the current batch.
        
        Returns:
            The loss for the current batch.
        """
        x, y = batch
        logits = self(x)
        loss = self.criterion(logits, y)
        
        # Log the metrics.
        self.log('test_loss', loss, on_epoch=True)
        self.log('test_acc', self.accuracy(logits, y), on_epoch=True,
                 prog_bar=True)
        
    def validation_step(self, batch, batch_idx):
        """A single validation step.

        Since we don't have a seperate validation set we just use the test
        logic.
        
        Args:
            batch (List(torch.LongTensor, torch.LongTensor)): The current batch
                inputs x and sentiment predictions y from the *val* dataloader.
            batch_idx (int): The index of the current batch.
        
        Returns:
            The loss for the current batch.
        """
        return self.test_step(batch, batch_idx)

### Train RNN Classification Model

In [ ]:
NUM_WORDS = 10000
MAX_SEQ_LEN = 128
EMBEDDING_DIM = 512
BATCH_SIZE = 32

imdb_data = IMDBDataModule(batch_size=BATCH_SIZE)
imdb_data.setup(num_words=NUM_WORDS,
                max_seq_len=MAX_SEQ_LEN)

model = TextClassificationRNN(d=EMBEDDING_DIM,
                              max_seq_len=MAX_SEQ_LEN,
                              num_tokens=NUM_WORDS)
logger = pl.loggers.TensorBoardLogger('tb_logs', name='rnn')
trainer = pl.Trainer(max_epochs=5,
                     default_root_dir='ckpts',
                     gpus=1,
                     logger=logger)
trainer.fit(model, imdb_data)
_ = trainer.test()

## Tensorboard

In [ ]:
%tensorboard --logdir tb_logs